In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

### Dig into to GPU pytorch tensor

In [2]:
import copy

import os
import sys
import torch
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import seaborn as sns
from collections import defaultdict


In [3]:
use_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(use_device)
'''Trivial data'''
edge_index = torch.tensor([[0, 1, 1, 3, 1, 2, 4, 2, 4, 6, 6, 7, 7, 9, 2, 5, 9, 8], 
                           [1, 0, 3, 1, 2, 1, 2, 4, 6, 4, 7, 6, 9, 7, 5, 2, 8, 9]])
# features = torch.rand(10, 3)
features = torch.tensor([[0, 0], [0, 1], [0, 2], [0, 3], [0, 4],  
                           [0, 5], [0, 6], [0, 7], [0, 8], [0, 9]], dtype = torch.float)
# label = torch.tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

label = torch.tensor([0, 1, 1, 0, 1, 1, 1, 0, 0, 0])

cuda


In [5]:
trial = features + torch.tensor([2, 4], dtype = torch.float)
print(trial)

tensor([[ 2.,  4.],
        [ 2.,  5.],
        [ 2.,  6.],
        [ 2.,  7.],
        [ 2.,  8.],
        [ 2.,  9.],
        [ 2., 10.],
        [ 2., 11.],
        [ 2., 12.],
        [ 2., 13.]])


In [9]:
# select the first dimension number
x = features.size(0)
print(x)

10


In [10]:
gpu_features = features.to(use_device)
print(gpu_features, gpu_features.shape)
part_gpu_features = gpu_features[[1, 3, 5], :]
print(part_gpu_features)

tensor([[0., 0.],
        [0., 1.],
        [0., 2.],
        [0., 3.],
        [0., 4.],
        [0., 5.],
        [0., 6.],
        [0., 7.],
        [0., 8.],
        [0., 9.]], device='cuda:0') torch.Size([10, 2])
tensor([[0., 1.],
        [0., 3.],
        [0., 5.]], device='cuda:0')


In [14]:
### modify the tensor in-place by indexing, need to make the dtype and the device both consistent
print(gpu_features, gpu_features.shape)
gpu_features[[1, 3, 5], :] = torch.tensor([[0, 11.0], [0, 13], [0, 15]], dtype = torch.float).to(use_device)
print(gpu_features, gpu_features.shape)

tensor([[0., 0.],
        [0., 1.],
        [0., 2.],
        [0., 3.],
        [0., 4.],
        [0., 5.],
        [0., 6.],
        [0., 7.],
        [0., 8.],
        [0., 9.]], device='cuda:0') torch.Size([10, 2])
tensor([[ 0.,  0.],
        [ 0., 11.],
        [ 0.,  2.],
        [ 0., 13.],
        [ 0.,  4.],
        [ 0., 15.],
        [ 0.,  6.],
        [ 0.,  7.],
        [ 0.,  8.],
        [ 0.,  9.]], device='cuda:0') torch.Size([10, 2])


In [22]:
gpu_features = features.to(use_device)
print(gpu_features, gpu_features.shape)
gpu_features[[1, 3, 5], :] = gpu_features[[2, 4, 6], :]
gpu_features = torch.cat([gpu_features, gpu_features[-2:].to(use_device) ])
print(gpu_features)

tensor([[0., 0.],
        [0., 1.],
        [0., 2.],
        [0., 3.],
        [0., 4.],
        [0., 5.],
        [0., 6.],
        [0., 7.],
        [0., 8.],
        [0., 9.]], device='cuda:0') torch.Size([10, 2])
tensor([[0., 0.],
        [0., 2.],
        [0., 2.],
        [0., 4.],
        [0., 4.],
        [0., 6.],
        [0., 6.],
        [0., 7.],
        [0., 8.],
        [0., 9.],
        [0., 8.],
        [0., 9.]], device='cuda:0')


### Manimpulate local-global index mapping by using torch.tensor

#### 1) Got the first remapping index tensor of all edges index (local) and copy it on to GPU

In [5]:
target_edges = [(109, 101), (115, 105), (102, 111), (104, 106), (111, 119)]
target_nodes = {left for left, right in target_edges} | {right for left, right in target_edges}
mapper = {node : i for i, node in enumerate(target_nodes)}
local_target = [ [mapper[start], mapper[end]] for start, end in target_edges]
print('remapped edges are: ')
print(local_target)
print('nodes on edges are: ')
print(target_nodes)
print('show the mapper: ')
print(mapper)

remapped edges are: 
[[5, 0], [7, 3], [1, 6], [2, 4], [6, 8]]
nodes on edges are: 
{101, 102, 104, 105, 106, 109, 111, 115, 119}
show the mapper: 
{101: 0, 102: 1, 104: 2, 105: 3, 106: 4, 109: 5, 111: 6, 115: 7, 119: 8}


In [6]:
# # second way through numpy array , not necessary
# target_np = np.array(local_target)
# print(target_np, type(target_np), target_np.shape)
# target_tsr = torch.from_numpy(target_np).to('cuda')

In [7]:

target_tsr = torch.LongTensor(local_target).to('cuda')
print(target_tsr, type(target_tsr), target_tsr.shape)

tensor([[5, 0],
        [7, 3],
        [1, 6],
        [2, 4],
        [6, 8]], device='cuda:0') <class 'torch.Tensor'> torch.Size([5, 2])


#### 2) On GPU, add the other direction and self-loops

In [8]:
target_tsr = target_tsr.t()
start, end = target_tsr
start, end = start.unsqueeze(0), end.unsqueeze(0)
print(start, type(start), start.shape)
# print()
print(end, type(end), end.shape)


tensor([[5, 7, 1, 2, 6]], device='cuda:0') <class 'torch.Tensor'> torch.Size([1, 5])
tensor([[0, 3, 6, 4, 8]], device='cuda:0') <class 'torch.Tensor'> torch.Size([1, 5])


In [9]:
target_rever = torch.cat([end, start], dim=0)
print(target_rever, type(target_rever), target_rever.shape)

tensor([[0, 3, 6, 4, 8],
        [5, 7, 1, 2, 6]], device='cuda:0') <class 'torch.Tensor'> torch.Size([2, 5])


In [10]:
target_comb = torch.cat([target_tsr, target_rever], dim=1)
print(target_comb, type(target_comb), target_comb.shape)

tensor([[5, 7, 1, 2, 6, 0, 3, 6, 4, 8],
        [0, 3, 6, 4, 8, 5, 7, 1, 2, 6]], device='cuda:0') <class 'torch.Tensor'> torch.Size([2, 10])


In [11]:
loop_index = torch.arange(0, len(target_nodes), dtype=target_tsr.dtype, device=target_tsr.device)
# loop_index = torch.LongTensor(list(mapper.values())).to('cuda')
loop_index = loop_index.unsqueeze(0).repeat(2, 1)
print(loop_index)

tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8],
        [0, 1, 2, 3, 4, 5, 6, 7, 8]], device='cuda:0')


In [12]:
target_final = torch.cat([target_comb, loop_index], dim=1)
print(target_final, type(target_final), target_final.shape)

tensor([[5, 7, 1, 2, 6, 0, 3, 6, 4, 8, 0, 1, 2, 3, 4, 5, 6, 7, 8],
        [0, 3, 6, 4, 8, 5, 7, 1, 2, 6, 0, 1, 2, 3, 4, 5, 6, 7, 8]],
       device='cuda:0') <class 'torch.Tensor'> torch.Size([2, 19])


### Investigate the double direction weights

<font color=orange>
In the two direction's of the same edge in the undirect-graph, the weights values are the same



In [29]:
import math
import random

from torch.nn import Parameter
from torch_scatter import scatter_add
# from torch_geometric.nn import MessagePassing
import torch.nn.functional as F
# from torch_geometric.nn import GCNConv
from torch_geometric.utils import add_remaining_self_loops

def get_edge_weight(edge_index, num_nodes, edge_weight=None, improved=False, dtype=None):
        
        if edge_weight is None:
            edge_weight = torch.ones((edge_index.size(1), ), dtype=dtype, device=edge_index.device)
        
        fill_value = 1 if not improved else 2
        # edge_index is already double direction if undirect, then add num_nodes self-loop edges added after the edge_index
        edge_index, edge_weight = add_remaining_self_loops(edge_index, edge_weight, fill_value, num_nodes)
        # after this edge_index is a 2 by (edge_num + node_num) tensor
        row, col = edge_index   
        # row includes the starting points of the edges  (first row of edge_index)
        # col includes the ending points of the edges   (second row of edge_index)

        deg = scatter_add(edge_weight, row, dim=0, dim_size=num_nodes)
        # row records the source nodes, which is the index we are trying to add
        # deg will record the out-degree of each node of x_i in all edges (x_i, x_j) including self_loops
        
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        normalized_edge_weight = deg_inv_sqrt[row] * edge_weight * deg_inv_sqrt[col]
        
        edge_index_global_self_loops = edge_index
        # transfer from tensor to the numpy to construct the dict for the edge_weights
        edge_index = edge_index.t().numpy()
        normalized_edge_weight = normalized_edge_weight.numpy()
        num_edge, _ = edge_index.shape
        # this info can also be stored as matrix considering the memory, depends whether the matrix is sparse or not
        edge_weight_global_dict = {(edge_index[i][0], edge_index[i][1]) : normalized_edge_weight[i] for i in range(num_edge)}
        
#         print('after adding self-loops, edge_index is', edge_index)
        edge_weight_global = [ edge_weight_global_dict[(edge[0], edge[1])] for edge in edge_index ]
    
        return edge_weight_global_dict

In [30]:
### Use trivial data

edge_index = torch.tensor([[0, 1, 1, 3, 1, 2, 4, 2, 4, 6, 6, 7, 7, 9, 2, 5, 9, 8], 
                           [1, 0, 3, 1, 2, 1, 2, 4, 6, 4, 7, 6, 9, 7, 5, 2, 8, 9]])
# features = torch.rand(10, 3)
features = torch.tensor([[0, 0], [0, 1], [0, 2], [0, 3], [0, 4],  
                           [0, 5], [0, 6], [0, 7], [0, 8], [0, 9]], dtype = torch.float)
# label = torch.tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

label = torch.tensor([0, 1, 1, 0, 1, 1, 1, 0, 0, 0])

In [31]:
edge_weight_global_dict = get_edge_weight(edge_index, features.shape[0])
print(edge_weight_global_dict)

{(0, 1): 0.35355338, (1, 0): 0.35355338, (1, 3): 0.35355338, (3, 1): 0.35355338, (1, 2): 0.25, (2, 1): 0.25, (4, 2): 0.28867513, (2, 4): 0.28867513, (4, 6): 0.3333333, (6, 4): 0.3333333, (6, 7): 0.3333333, (7, 6): 0.3333333, (7, 9): 0.3333333, (9, 7): 0.3333333, (2, 5): 0.35355338, (5, 2): 0.35355338, (9, 8): 0.40824828, (8, 9): 0.40824828, (0, 0): 0.49999997, (1, 1): 0.25, (2, 2): 0.25, (3, 3): 0.49999997, (4, 4): 0.3333333, (5, 5): 0.49999997, (6, 6): 0.3333333, (7, 7): 0.3333333, (8, 8): 0.49999997, (9, 9): 0.3333333}


### Create double_add_self_loop edge weights by a single direction edge weight

In [33]:
tmp = edge_index.t().numpy().tolist()
graph = nx.from_edgelist(tmp)

test_edges = {tuple(sorted(edge)) for edge in graph.edges()}
test_nodes = sorted(node for node in graph.nodes())
    
print(test_edges)
test_edge_weight_local = [ edge_weight_global_dict[(edge[0], edge[1])] for edge in test_edges ]       
test_edge_weight_selfloop_local = [ edge_weight_global_dict[(i, i)] for i in test_nodes ]

{(0, 1), (1, 2), (1, 3), (6, 7), (4, 6), (8, 9), (2, 5), (2, 4), (7, 9)}


In [35]:
test_edge_weight_local_tsr = torch.FloatTensor(test_edge_weight_local).to('cuda')
test_edge_weight_selfloop_local_tsr = torch.FloatTensor(test_edge_weight_selfloop_local).to('cuda')
print(test_edge_weight_local_tsr)
print(test_edge_weight_selfloop_local_tsr)

tensor([0.3536, 0.2500, 0.3536, 0.3333, 0.3333, 0.4082, 0.3536, 0.2887, 0.3333],
       device='cuda:0')
tensor([0.5000, 0.2500, 0.2500, 0.5000, 0.3333, 0.5000, 0.3333, 0.3333, 0.5000,
        0.3333], device='cuda:0')


In [38]:
edge_weights = torch.cat([test_edge_weight_local_tsr, test_edge_weight_local_tsr, test_edge_weight_selfloop_local_tsr], dim=0)
print(edge_weights, edge_weights.shape)

tensor([0.3536, 0.2500, 0.3536, 0.3333, 0.3333, 0.4082, 0.3536, 0.2887, 0.3333,
        0.3536, 0.2500, 0.3536, 0.3333, 0.3333, 0.4082, 0.3536, 0.2887, 0.3333,
        0.5000, 0.2500, 0.2500, 0.5000, 0.3333, 0.5000, 0.3333, 0.3333, 0.5000,
        0.3333], device='cuda:0') torch.Size([28])


### Implement the overlap between different train-batches (nodes and edges)

In [47]:
a = [3, 4, 5]
b = [6, 7, 8]
for i in range(1, len(a)):
    for left, right in zip(a, a[i:]):
        print(left, right)
        
a = {3, 4, 5}; b = {7, 8, 6}
c = a & b
print(c, type(c), len(c))

3 4
4 5
3 5
set() <class 'set'> 0
